# Homework 1

## Instructions

First, ensure you have cloned the [course repository](https://github.com/lydiaYchen/DDL25Spring).

Then, open the [interactive notebook version](https://github.com/lydiaYchen/DDL25Spring/blob/main/lab/homework-1.ipynb) of this homework from your local copy.

For part A, fill in the code and answers within the notebook and save your changes.

For part B, create and archive the necessary Python/shell scripts together.

Finally, upload the notebook and the archive to the assignment in ILIAS.

## Part A (12 points)

### Note

When not otherwise specified, use the following parameter values in experiment runs:
- `nr_clients` (N): 100
- `lr`: 0.01
- `client_fraction` (C): 0.1
- `nr_local_epochs` (E): 1
- `batch_size` (B): 100
- `nr_rounds`: 10
- `iid`: True

For all exercises, pass `seed = 10` to calls for splitting data, server initialization, or plotting.

### Exercise A1: FedSGD with weights (3 points)

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from hfl_complete import *

n = 100
lr = 0.01
c = 0.1
e = 1
b = 100
nr_rounds = 10
iid = True
seed = 10

plt.style.use("ggplot")

#### Question

_(2 points)_ Implement a version of FedSGD that uses weights in its updates, like FedAvg, instead of the gradients from the version of the tutorials. The two FedSGD versions should have the same test accuracy after each round (with a tolerance of at most 0.02%). To show this, compare their output for the following two scenarios over *5 rounds*:
- `lr = 0.01, client_subsets = split(100, True, ...), client_fraction = 0.5`
- `lr = 0.1, client_subsets = split(50, False, ...), client_fraction = 0.2`

*Tip:* You can use the existing FedAvg implementation to minimize the amount of code writing required.

_(1 point)_ Explain in which cases (about the different parameters for decentralized learning) the two are equivalent.

#### Answer

In [ ]:
from tqdm import tqdm

class FedSgdWeightServer(DecentralizedServer):
    def __init__(
        self,
        lr: float,
        client_subsets: list[Subset],
        client_fraction: float,
        seed: int
    ) -> None:
        super().__init__(lr, -1, client_subsets, client_fraction, seed)
        # no optimizer: we train locally this time
        self.name = "FedSGDWeight"
        # like in FedSGD, only 1 epoch
        self.nr_local_epochs = 1
        # we can reuse the clients from the FedAvg; just train on the full mini-batch for 1 epoch
        self.clients = [
            WeightClient(subset, lr, len(subset), self.nr_local_epochs)
            for subset in client_subsets
        ]


    def run(self, nr_rounds: int) -> RunResult:
        elapsed_time = 0.0
        run_result = RunResult(
            self.name,
            self.nr_clients,
            self.client_fraction,
            self.batch_size, # -1
            self.nr_local_epochs, # 1
            self.lr,
            self.seed,
        )

        for nr_round in tqdm(range(nr_rounds), desc="Rounds", leave=False):
            setup_start_time = perf_counter()
            self.model.train()
            weights = [x.detach().cpu().clone() for x in self.model.parameters()]
            indices_chosen_clients = self.rng.choice(
                self.nr_clients, self.nr_clients_per_round, replace=False
            )
            chosen_sum_nr_samples = sum(
                self.client_sample_counts[i] for i in indices_chosen_clients
            )
            chosen_adjusted_weights: list[list[torch.Tensor]] = []
            elapsed_time += perf_counter() - setup_start_time
            update_time = 0.0

            for c_i in indices_chosen_clients:
                update_start_time = perf_counter()
                ind = int(c_i)
                client_round_seed = (
                    self.seed + ind + 1 + nr_round * self.nr_clients_per_round
                )
                client_weights = self.clients[ind].update(weights, client_round_seed)
                chosen_adjusted_weights.append(
                    [
                        self.client_sample_counts[ind] / chosen_sum_nr_samples * tens
                        for tens in client_weights
                    ]
                )
                update_time = max(update_time, perf_counter() - update_start_time)

            elapsed_time += update_time
            aggregate_start_time = perf_counter()
            averaged_chosen_weights: list[torch.Tensor] = [
                torch.stack(x, dim=0).sum(dim=0) for x in zip(*chosen_adjusted_weights)
            ]

            with torch.no_grad():
                zip_parameter_weight = zip(
                    self.model.parameters(), averaged_chosen_weights
                )
                for server_parameter, client_weight in zip_parameter_weight:
                    server_parameter[:] = client_weight.to(device=device)

            elapsed_time += perf_counter() - aggregate_start_time
            run_result.wall_time.append(round(elapsed_time, 1))
            run_result.message_count.append(
                2 * (nr_round + 1) * self.nr_clients_per_round
            )
            run_result.test_accuracy.append(self.test())

        return run_result

#### Scenario 1

In [ ]:
# scenario 1 FedSGD (Weights)
lr = 0.01
client_subsets = split(100, True, seed=seed)
client_fraction = 0.5

fedsgd_weight_server = FedSgdWeightServer(
    lr=lr, client_subsets=client_subsets, client_fraction=client_fraction, seed=seed
)
# run for 5 rounds
results_fedsgd_weight = fedsgd_weight_server.run(5)
fedsgd_weight_df = results_fedsgd_weight.as_df()
fedsgd_weight_df

In [ ]:
# scenario 1: FedSGD (Gradient)
lr = 0.01
client_subsets = split(100, True, seed=seed)
client_fraction = 0.5

fedsgd_gradient_server = FedSgdGradientServer(
    lr=lr, client_subsets=client_subsets, client_fraction=client_fraction, seed=seed
)
# run for 5 rounds
result_fedsgd_gradient = fedsgd_gradient_server.run(5)
fedsgd_gradient_df = result_fedsgd_gradient.as_df()
fedsgd_gradient_df

#### Scenario 2

In [ ]:
# scenario 2: FedSGD (Weights)
lr = 0.1
client_subsets = split(50, False, seed=seed)
client_fraction = 0.2

fedsgd_weight_server = FedSgdWeightServer(
    lr=lr, client_subsets=client_subsets, client_fraction=client_fraction, seed=seed
)
# run for 5 rounds
results_fedsgd_weight = fedsgd_weight_server.run(5)
fedsgd_weight_df = results_fedsgd_weight.as_df()
fedsgd_weight_df

In [ ]:
# scenario 2: FedSGD (Gradient)
lr = 0.1
client_subsets = split(50, False, seed=seed)
client_fraction = 0.2

fedsgd_gradient_server = FedSgdGradientServer(
    lr=lr, client_subsets=client_subsets, client_fraction=client_fraction, seed=seed
)

# run for 5 rounds
result_fedsgd_gradient = fedsgd_gradient_server.run(5)
fedsgd_gradient_df = result_fedsgd_gradient.as_df()
fedsgd_gradient_df

*Q: (1 point) Explain in which cases (about the different parameters for decentralized learning) the two are equivalent.*

In FedSGD, we do the forward pass on the full batch (-1), compute gradients, and send the gradients to the server.
The server then averages the gradients and does the optimization step. On the next run, it will send the new weights to the clients.
In FedAvg, each client trains for n epoch on mini-batches (forward, backward, update). The weights after training are sent to the server. The server averages the weights. On the next run, it will send the new weights to the clients.

In FedSGDWeight, we do the forward pass on the full batch, compute the gradients, do the optimization step and send back the weights to the clients.
This can be seen as a FedSGD with local weight update, or a FedAvg with 1 epoch and no mini-batches.
If we were to do FedSGDWeight with multiple Epochs, or with mini-batches, then this would become a FedAvg.

### Exercise A2: Client number & fraction (4 points)

#### Question

_(2 points)_ Run the necessary experiments to fill in the following table showing the final message count and test accuracy of FedSGD and FedAvg for different total client numbers:

| Algorithm | N   | C   | Message count | Test accuracy |
| --------- | --- | --- | ------------- | ------------- |
| FedSGD    | 10  | 0.1 | 20            | 42.87         |
| FedAvg    | 10  | 0.1 | 20            | 93.2          |
| FedSGD    | 50  | 0.1 | 100           | 43.43         |
| FedAvg    | 50  | 0.1 | 100           | 87.71         |
| FedSGD    | 100 | 0.1 | 200           | 42.74         |
| FedAvg    | 100 | 0.1 | 200           | 80.89         |

Is the relationship between the metrics and client numbers monotonous?

_(2 points)_ Run the experiments to fill in the table when varying the fraction of clients used in every round:

| Algorithm | N   | C    | Message count | Test accuracy |
| --------- | --- | ---- | ------------- | ------------- |
| FedSGD    | 100 | 0.01 | 20            | 41.01         |
| FedAvg    | 100 | 0.01 | 20            | 75.47         |
| FedSGD    | 100 | 0.1  | 200           | 42.74         |
| FedAvg    | 100 | 0.1  | 200           | 80.89         |
| FedSGD    | 100 | 0.2  | 400           | 42.68         |
| FedAvg    | 100 | 0.2  | 400           | 81.64         |

How does the observed pattern differ?

#### Answer

In [ ]:
n = 100        # nr_clients (N)
lr = 0.01      # learning rate (lr)
c = 0.1        # client fraction (C)
e = 1          # nr local epochs (E)
b = 100        # batch size (B)
nr_rounds = 10 # nr rounds
iid = True     # independent and identically distributed
seed = 10      # seed for data generation

In [ ]:
ex_a21_results = []

c = 0.1
for n in [10, 50, 100]:
    client_subsets = split(n, iid=iid, seed=seed)
    # FedSGD
    fedsgd_gradient_server = FedSgdGradientServer(
        lr=lr, client_subsets=client_subsets, client_fraction=c, seed=seed
    )
    result_fedsgd_gradient = fedsgd_gradient_server.run(nr_rounds=nr_rounds)
    ex_a21_results.append(result_fedsgd_gradient.as_df().iloc[-1])

    # FedAvg
    fedavg_server = FedAvgServer(
        lr=lr, batch_size=b, client_subsets=client_subsets, client_fraction=c, nr_local_epochs=e, seed=seed
    )
    result_fedavg = fedavg_server.run(nr_rounds=nr_rounds)
    ex_a21_results.append(result_fedavg.as_df().iloc[-1])

ex_a21_df = pd.concat(ex_a21_results, axis=1, ignore_index=True).T
ex_a21_df.loc[: , ["Algorithm", "N", "C", "Message count", "Test accuracy"]]

In [ ]:
ex_a22_results = []

n = 100
for c in [0.01, 0.1, 0.2]:
    client_subsets = split(n, iid=iid, seed=seed)
    # FedSGD
    fedsgd_gradient_server = FedSgdGradientServer(
        lr=lr, client_subsets=client_subsets, client_fraction=c, seed=seed
    )
    result_fedsgd_gradient = fedsgd_gradient_server.run(nr_rounds=nr_rounds)
    ex_a22_results.append(result_fedsgd_gradient.as_df().iloc[-1])

    # FedAvg
    fedavg_server = FedAvgServer(
        lr=lr, batch_size=b, client_subsets=client_subsets, client_fraction=c, nr_local_epochs=e, seed=seed
    )
    result_fedavg = fedavg_server.run(nr_rounds=nr_rounds)
    ex_a22_results.append(result_fedavg.as_df().iloc[-1])

ex_a22_df = pd.concat(ex_a22_results, axis=1, ignore_index=True).T
ex_a22_df.loc[: , ["Algorithm", "N", "C", "Message count", "Test accuracy"]]

### Exercise A3: Local epoch count & (non-)IID data (5 points)

#### Question

_(1 point)_ Create a line plot of the accuracy after each round for the following algorithm variants:

- FedSGD
- FedAvg (E=1)
- FedAvg (E=2)
- FedAvg (E=4)

How does FedAvg compare to FedSGD? What is the effect of increasing the work clients perform locally for each update in FedAvg?

_(2 points)_ Make one line plot of FedSGD and FedAvg under an IID and non-IID split for 15 rounds (leaving all other parameter values as they previously mentioned default). How does the non-IID setting affect the accuracy achieved by the two algorithms? What is the difference in terms of the smoothness of learning?

_(2 points)_ Make another plot for only non-IID splits, including the FedSGD and FedAvg configs from before, and add a version for each with a learning rate of 0.001 and client fraction of 0.5. How does the stability of the new variants compare to the old ones? Why do the changes in parameters have the observed effect?

#### Answer 1: FedAvg vs FedSGD

In [ ]:
n = 100        # nr_clients (N)
lr = 0.01      # learning rate (lr)
c = 0.1        # client fraction (C)
e = 1          # nr local epochs (E)
b = 100        # batch size (B)
nr_rounds = 10 # nr rounds
iid = True     # independent and identically distributed
seed = 10      # seed for data generation

ex_a3_results = []

# FedSGD
client_subsets = split(nr_clients=n, iid=iid, seed=seed)

fedsgd_gradient_server = FedSgdGradientServer(
    lr=lr, client_subsets=client_subsets, client_fraction=c, seed=seed
)
result_fedsgd_gradient = fedsgd_gradient_server.run(nr_rounds=nr_rounds)
ex_a3_results.append(result_fedsgd_gradient.as_df())


# FedAvg (E = 1)
client_subsets = split(nr_clients=n, iid=iid, seed=seed)

fedavg_server = FedAvgServer(
    lr=lr, batch_size=b, client_subsets=client_subsets, client_fraction=c, nr_local_epochs=1, seed=seed
)
result_fedavg = fedavg_server.run(nr_rounds=nr_rounds)
ex_a3_results.append(result_fedavg.as_df())


# FedAvg (E = 2)
client_subsets = split(nr_clients=n, iid=iid, seed=seed)

fedavg_server = FedAvgServer(
    lr=lr, batch_size=b, client_subsets=client_subsets, client_fraction=c, nr_local_epochs=2, seed=seed
)
result_fedavg_2 = fedavg_server.run(nr_rounds=nr_rounds)
ex_a3_results.append(result_fedavg_2.as_df())


# FedAvg (E = 4)
client_subsets = split(nr_clients=n, iid=iid, seed=seed)

fedavg_server = FedAvgServer(
    lr=lr, batch_size=b, client_subsets=client_subsets, client_fraction=c, nr_local_epochs=4, seed=seed
)
result_fedavg_4 = fedavg_server.run(nr_rounds=nr_rounds)
ex_a3_results.append(result_fedavg_4.as_df())

In [ ]:
# Combined Plot

df = pd.concat(ex_a3_results, ignore_index=True)
df["Algorithm"] = df["Algorithm"] + f" (E={df["E"].astype(str)})"

fig, ax = plt.subplots(1, 1, figsize=(9, 6))

sns.lineplot(df, x="Round", y="Test accuracy", hue="Algorithm", seed=seed, ax=ax)
ax.set_xticks(df["Round"].unique())
ax.set_yticks(np.arange(0, 101, 10));

Q: *How does FedAvg compare to FedSGD? What is the effect of increasing the work clients perform locally for each update in FedAvg?*

FedAvg performs much better than FedSGD, even when doing only 1 Epoch.
At each round of FedAvg, we are training on multiple clients for $E = \{1, 2, 4\}$ Epochs, then send the weights to update the model.
Meanwhile, in FedSGD, we only do 1 round and send the gradients instead.

Between the FedAvg runs, we see that a higher number of Epochs, resulting in more local work by the clients, leads to a better accuracy score.
With an Epoch of 2 and 4, we obtain very close test accuracy at 10 rounds.
However the clients must do twice the amount of work in the latter configuration.
An Epoch of 2 leads to a increase of almost 10 percentage points, while only losing a small amount of accuracy compared to 4 Epochs.


#### Answer 2: FedSGD vs FedAvg under an IID and non-IID split (15 rounds)

In [ ]:
n = 100        # nr_clients (N)
lr = 0.01      # learning rate (lr)
c = 0.1        # client fraction (C)
e = 1          # nr local epochs (E)
b = 100        # batch size (B)
nr_rounds = 10 # nr rounds
iid = True     # independent and identically distributed
seed = 10      # seed for data generation

# FedSGD with iid = True
ex_a3_fedsgd_results = []
client_subsets = split(nr_clients=n, iid=True, seed=seed)

fedsgd_gradient_server = FedSgdGradientServer(
    lr=lr, client_subsets=client_subsets, client_fraction=c, seed=seed
)
result_fedsgd_gradient = fedsgd_gradient_server.run(nr_rounds=15)
fedsgd_df = result_fedsgd_gradient.as_df()

fedsgd_df["Algorithm"] = fedsgd_df["Algorithm"] + f" (iid={True})"
ex_a3_fedsgd_results.append(fedsgd_df)


# FedSGD with iid = False
client_subsets = split(nr_clients=n, iid=False, seed=seed)

fedsgd_gradient_server = FedSgdGradientServer(
    lr=lr, client_subsets=client_subsets, client_fraction=c, seed=seed
)
result_fedsgd_gradient = fedsgd_gradient_server.run(nr_rounds=15)
fedsgd_df = result_fedsgd_gradient.as_df()
fedsgd_df["Algorithm"] = fedsgd_df["Algorithm"] + f" (iid={False})"
ex_a3_fedsgd_results.append(fedsgd_df)


# FedAvg with iid = True
ex_a3_fedavg_results = []
client_subsets = split(nr_clients=n, iid=True, seed=seed)

fedavg_server = FedAvgServer(
    lr=lr, batch_size=b, client_subsets=client_subsets, client_fraction=c, nr_local_epochs=e, seed=seed
)
result_fedavg = fedavg_server.run(nr_rounds=15)
fedavg_df = result_fedavg.as_df()
fedavg_df["Algorithm"] = fedavg_df["Algorithm"] + f" (iid={True})"
ex_a3_fedavg_results.append(fedavg_df)

# FedAvg with iid = False
client_subsets = split(nr_clients=n, iid=False, seed=seed)

fedavg_server = FedAvgServer(
    lr=lr, batch_size=b, client_subsets=client_subsets, client_fraction=c, nr_local_epochs=e, seed=seed
)
result_fedavg = fedavg_server.run(nr_rounds=15)
fedavg_df = result_fedavg.as_df()
fedavg_df["Algorithm"] = fedavg_df["Algorithm"] + f" (iid={False})"
ex_a3_fedavg_results.append(fedavg_df)

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(12, 6))

ex_a3_fedsgd_df = pd.concat(ex_a3_fedsgd_results)
sns.lineplot(ex_a3_fedsgd_df, x="Round", y="Test accuracy", hue="Algorithm", seed=seed, ax=ax1)

ax1.set_xticks(ex_a3_fedsgd_df["Round"].unique())
ax1.set_yticks(np.arange(0, 101, 10))
ax1.set_title("FedSGD (IID/non-IID)")

ex_a3_fedavg_df = pd.concat(ex_a3_fedavg_results)
sns.lineplot(ex_a3_fedavg_df, x="Round", y="Test accuracy", hue="Algorithm", seed=seed, ax=ax2)

ax2.set_xticks(ex_a3_fedavg_df["Round"].unique())
ax2.set_yticks(np.arange(0, 101, 10))
ax2.set_title("FedAvg (IID/non-IID)");

 Q: *How does the non-IID setting affect the accuracy achieved by the two algorithms? What is the difference in terms of the smoothness of learning?*



#### Answer 3: FedSGD vs FedAvg non-IID splits, with varying learning rates and client fractions

 (2 points) Make another plot for only non-IID splits, including the FedSGD and FedAvg configs from before, and add a version for each with a learning rate of 0.001 and client fraction of 0.5. How does the stability of the new variants compare to the old ones? Why do the changes in parameters have the observed effect?

In [ ]:
n = 100        # nr_clients (N)
lr = 0.01      # learning rate (lr)
c = 0.1        # client fraction (C)
e = 1          # nr local epochs (E)
b = 100        # batch size (B)
nr_rounds = 10 # nr rounds
iid = True     # independent and identically distributed
seed = 10      # seed for data generation

ex_a3_fedsgd_non_idd_results = []
client_subsets = split(nr_clients=n, iid=False, seed=seed)

# config 1: default lr and default fraction
fedsgd_gradient_server = FedSgdGradientServer(
    lr=lr, client_subsets=client_subsets, client_fraction=c, seed=seed
)
fedsgd_df = fedsgd_gradient_server.run(nr_rounds=15).as_df()

#ex_a3_fedsgd_non_idd_results.append()

fedsgd_df["Algorithm"] + f"(lr={df[""]})"

In [ ]:

# config 2: lr=0.001, fraction=0.5
fedsgd_gradient_server = FedSgdGradientServer(
    lr=0.001, client_subsets=client_subsets, client_fraction=0.5, seed=seed
)
ex_a3_fedsgd_non_idd_results.append(fedsgd_gradient_server.run(nr_rounds=15).as_df())

## FedAvg with iid = False
ex_a3_fedavg_non_idd_results = []

client_subsets = split(nr_clients=n, iid=False, seed=seed)

# config 1: default lr and default fraction
fedavg_server = FedAvgServer(
    lr=lr, batch_size=b, client_subsets=client_subsets, client_fraction=c, nr_local_epochs=e, seed=seed
)
ex_a3_fedavg_non_idd_results.append(fedavg_server.run(nr_rounds=15).as_df())

# config 2: lr=0.001, fraction=0.5
fedavg_server = FedAvgServer(
    lr=0.001, batch_size=b, client_subsets=client_subsets, client_fraction=0.5, nr_local_epochs=e, seed=seed
)
ex_a3_fedavg_non_idd_results.append(fedavg_server.run(nr_rounds=15).as_df())

In [ ]:
# Plotting the two non-IDD results
ex_a3_non_iid_df = pd.concat(ex_a3_non_iid_results, ignore_index=True)

fig, ax = plt.subplots(1, 1, figsize=(9, 6))

sns.lineplot(ex_a3_non_iid_df, x="Round", y="Test accuracy", hue="Algorithm", seed=seed, ax=ax)
ax.set_xticks(ex_a3_non_iid_df["Round"].unique())
ax.set_yticks(np.arange(0, 101, 10));

Q: *How does the stability of the new variants compare to the old ones? Why do the changes in parameters have the observed effect?*


## Part B (12 points)

### Exercise B1: Microbatch Pipeline Model Parallelism (7 points)

Implement pipeline parallelism with microbatches, as discussed during the lab.

As with the other data/model parallelism examples, you will need a Python script for the nodes and a shell script to orchestrate execution.

Be aware of the possibility of deadlocks: due to how `good` operates, it is possible to deadlock by having device 1 send $B_2$ to device 2 in the forward pass, and simultaneously, device 2 send $B_1$ in the backward pass.
Since both operations will await a corresponding receive the training will stop indefinitely.

Use `isend` & `irecv`, the asynchronous (non-blocking) versions of `send` & `recv` in `torch.distributed`.
Add comments or text explaining how you expect your implementation to work and test that it runs for the same number of steps and model architecture as in class.

Note that `torch.distributed`'s implementation of `gloo` does not currently support properly asynchronous communication even when using the corresponding primitives.
Thus, you will not see the same improvements in speed as with a backend like `nccl`.

You may also take advantage of the fact that `torch` gradients naturally accumulate if zeroed out.
Also, scaling the loss by a constant is equivalent to scaling the resulting gradients by the same constant.

### Exercise B2: Joint Data & Model Parallelism (5 points)

Implement a training setup that uses data and model parallelism together.

Create 2 pipelines of 3 stages running sequentially, where each stage works with 3 sequential micro-batches.

Once again, add comments or text explaining your implementation and test it on the setting that mimics those from the class.

You can use groups from `torch.distributed` to handle operations that require interaction between a subset of more than two but less than all workers.